# packages and imports

In [ ]:
pip install ray

In [ ]:
pip install xgboost

In [ ]:
pip install shap

In [ ]:
pip install interpret

In [1]:
#imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from statistics import mean

from data_makers import *
import utils
from utils import *
import mean_model
from mean_model import meanModel
import ML_models
from ML_models import *

import sklearn
from sklearn.preprocessing import OneHotEncoder
from sklearn.inspection import permutation_importance

from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_squared_error, r2_score

from sklearn.preprocessing import MinMaxScaler
from joblib import parallel_backend
from ray.util.joblib import register_ray

from xgboost import XGBRegressor

import heapq

import shap
from shap.explainers import Tree
from interpret.blackbox import ShapKernel

import interpret.glassbox
from interpret import show


In [ ]:
import importlib
importlib.reload(utils)

# Dataframe makers

## phospho

In [5]:
#read in the X dataframe
X_phos = pd.read_csv('data/X_phos', index_col=0)

#read in the y dataframe
y_phos = pd.read_csv('data/y_phos', index_col=0)

#one hot representations of drugs from y
hotdrugsDF_phos = one_hot_maker(y_phos)

#produce X-main and y_main
cl_phos = clMaker(X_phos, y_phos)
x_all_phos, x_drug_phos, y_main_phos = create_all_drugs(x=X_phos, xd=hotdrugsDF_phos, y=y_phos, cells=cl_phos)
X_main_phos = X_main_maker(x_all_phos, x_drug_phos, short = False)

## proteomic

In [ ]:
#read in the X dataframe
X_prot = pd.read_csv('data/X_prot', index_col=0)

#read in the y dataframe
y_prot = pd.read_csv('data/y_prot', index_col=0)

#dl maker 
dl_prot = dlMaker(y_prot)

#one hot representations of drugs from y
hotdrugsDF_prot = one_hot_maker(y_prot)

#produce X-main and y_main
cl_prot = clMaker(X_prot, y_prot)
x_all_prot, x_drug_prot, y_main_prot = create_all_drugs(x=X_prot, xd=hotdrugsDF_prot, y=y_prot, cells=cl_prot)
X_main_prot = X_main_maker(x_all_prot, x_drug_prot, short = False)

# first train-test split

N.B. X_train and y_train replace X_main and y_main

In [4]:
#regular split
X_train, X_test, y_train, y_test = cell_line_split(X_main_phos, y_main_phos, test_size=0.2, random_state = 0)

In [ ]:
#short split
X_train_short, X_test_short, y_train_short, y_test_short = cell_line_split(X_main[:1000], y_main[:1000], test_size=0.2, random_state = 0)

# Benchmark mean model 

create a model that predicts IC50 by looking at mean IC50 value for that drug

In [ ]:
dl = dlMaker(y, noRepeats=True)

In [ ]:
mm = meanModel(y_train, dl)

In [ ]:
prediction = mm.predict(y_test.index)

In [ ]:
r2_score(y_test, list(prediction.values()))

# Random Forest Regressor

In [ ]:
def rfr(X = X_train_short, y = y_train_short, test_size=0.1, random_state = 0, iterations = 5):
    r2_mean_list = []
    MSE_mean_list = []
    for i in range(iterations):
        print(f'Iteration: {i+1}')
        X_train, X_val, y_train, y_val = cell_line_split(X, y, test_size, random_state) #train-validation split
        classify = RandomForestRegressor(n_jobs=-1, max_depth=300, n_estimators=200)
        classify.fit(X_train.values, y_train)
        y_pred = classify.predict(X_val)
        r2 = r2_score(y_val, y_pred)
        MSE = mean_squared_error(y_val.values, y_pred)
        print(f'r2={r2}, MSE={MSE}')
        r2_mean_list.append(r2)
        MSE_mean_list.append(MSE)
    r2_mean = mean(r2_mean_list)
    MSE_mean = mean(MSE_mean_list)
    return r2_mean, MSE_mean

In [ ]:
def rfrFeatSelect(X = X_train_short, y = y_train_short, test_size=0.1, random_state = 0, iterations = 1):
    r2_mean_list = []
    MSE_mean_list = []
    for i in range(iterations):
        print(f'Iteration: {i+1}')
        X_train, X_val, y_train, y_val = cell_line_split(X, y, test_size, random_state) #train-validation split
        classify = RandomForestRegressor(n_jobs=-1, max_depth=300, n_estimators=200)
        classify.fit(X_train.values, y_train)
        y_pred = classify.predict(X_val.values)
        featSelect = classify.feature_importances_
        r2 = r2_score(y_val, y_pred)
        MSE = mean_squared_error(y_val, y_pred)
        print(f'r2={r2}, MSE={MSE}')
        r2_mean_list.append(r2)
        MSE_mean_list.append(MSE)
    r2_mean = mean(r2_mean_list)
    MSE_mean = mean(MSE_mean_list)
    return r2_mean, MSE_mean, featSelect

# Landmark genes

Makes a version of the X dataframe with only genes defined by the LINCS L1000 landmark gene paper

In [ ]:
#read in the LINCS data 
landmark_genes = pd.read_table("Landmark_genes_LINCS.txt")
landmarkGenes = [x for x in landmark_genes['Symbol']]

In [ ]:
# filter X for just landmark genes
X_L1000 = landmark_X_maker(X, landmarkGenes)

## feature selection for landmark genes

In [ ]:
#Make the ML inputs filtered for landmark genes
L1000 = landmark_X_maker(X, landmarkGenes)
hotdrugsDF_L1000 = one_hot_maker(y)
x_all_L1000, x_drug_L1000, y_main_L1000 = create_all_drugs(x=L1000, xd=hotdrugsDF_L1000, y=y, cells=cl)
X_main_L1000 = X_main_maker(x_all_L1000, x_drug_L1000, short = False)

In [ ]:
#train-test split
X_train_L1000, X_test_L1000, y_train_L1000, y_test_L1000 = cell_line_split(X_main_L1000, y_main_L1000, cl, test_size=0.2, random_state = 0)

In [ ]:
# run the models for random forest 
r2, MSE = rfr(X = X_train_L1000, y = y_train_L1000, iterations=10)
print('R-squared: '+str(r2)+'\n'+'Mean squared error: '+str(MSE))

In [ ]:
#create dataframes that take a random set of features with the same length as landmark as a comparison
random_set = X.sample(n=2056,axis='columns')
hotdrugsDF_rand = one_hot_maker(y)
x_all_rand, x_drug_rand, y_main_rand = create_all_drugs(x=random_set, xd=hotdrugsDF_rand, y=y, cells=cl)
X_main_rand = X_main_maker(x_all_rand, x_drug_rand, short = False)

#train-test split
X_train_rand, X_test_rand, y_train_rand, y_test_rand = cell_line_split(X_main_rand, y_main_rand, cl, test_size=0.2, random_state = 0)

# run the model
r2, MSE = rfr(X = X_train_rand, y = y_train_rand, iterations=10)
print('R-squared: '+str(r2)+'\n'+'Mean squared error: '+str(MSE))

# feature selection with rfr feature_importances

## feature selection

Here I run the models and store the feature_importances_ data

In [ ]:
#run the model that outputs feature_importances_ attribute with full length
r2, MSE, classify = rfrFeatSelect(X = X_main, y = y_main, random_state = 88, iterations=1)
print('R-squared: '+str(r2)+'\n'+'Mean squared error: '+str(MSE))

# classify is the feature_importances_ array
print(classify)

In [ ]:
#this function outputs the top x number of features and their scores for a model
rfr_final_names, rfr_final_scores = rfrFeatures(classify, X_main = X_main, topX = 10411, N = 10411)

In [ ]:
# assign plot name variable for saving plots
plot_name = 'plots/rfr_proteomic_rs88.png'

In [ ]:
#plot the feature_importances_

plt.rcParams['figure.figsize'] = [20, 20]
plt.plot(rfr_final_names[:30], rfr_final_scores[:30], linestyle='-', marker='.', color='#009d9a', linewidth=1)
rot = plt.xticks(rotation=45)
plt.savefig(plot_name)

In [ ]:
# creates a dictionary of features in order of importance with their score to store in a text file
rfrdict = {rfr_final_names[i]:rfr_final_scores[i] for i in range(len(rfr_final_names))}

In [ ]:
# specify text file name
file_name = 'feat_select_files/proteomic/rfr_feat_select_proteomic_rs88.txt'

In [ ]:
#store all feature_importance_ date in a text file

with open(file_name, "w") as txt_file:
    for key, value in rfrdict.items():
        txt_file.write(key +':'+ str(value) + "\n") 

In [ ]:
# Import data from the text files and make final_names/final_scores from feature txt file
rfr_final_names = []
rfr_final_scores = []
with open(file_name, "r") as features:
    lines = features.readlines()
    for i in lines:
        phospho = i.split(':')[0]
        score = i.split(':')[1]
        score = score.split("\n")[0]
        rfr_final_names.append(phospho)
        rfr_final_scores.append(float(score))

In [ ]:
#plot this data

plt.rcParams['figure.figsize'] = [20, 20]
plt.plot(rfr_final_names[:50], rfr_final_scores[:50], linestyle='-', marker='.', color='#009d9a', linewidth=1)
rot = plt.xticks(rotation=45)
plt.savefig('plots/rfr_1.png')

## feature selection testing

Test how effectively the model runs with various numbers of features from the top x feature_importances_ outputs

In [2]:
#read in the feature_importances_ feature selected data
feature_list = []
with open("feat_select_files/phospho/rfr_X_main/rfr_feat_select.txt", "r") as features:
    lines = features.readlines()
    for i in lines:
        i.replace(" ", "")
        feature_list.append(i.split(":")[0])

In [ ]:
seeds = [0, 8, 23, 42, 69, 88]
for i in range(5,10):
    index = i*100
    for s in range(len(seeds)):
        #create the dataframes
        X_features = X_phos.reindex(feature_list[:index],axis="columns")
        hotdrugsDF_feats = one_hot_maker(y_phos)
        cl_phos = clMaker(X_phos, y_phos)
        x_all_feats, x_drug_feats, y_main_feats = create_all_drugs(x=X_features, xd=hotdrugsDF_feats, y=y_phos, cells=cl_phos)
        X_main_feats = X_main_maker(x_all_feats, x_drug_feats, short = False)
        X_train_features, X_test_features, y_train_features, y_test_features = cell_line_split(X_main_feats, y_main_feats, test_size=0.2, random_state = seeds[s])
        
        #status update print statements
        print(f'number of features: {index}, seed: {seeds[s]}')
        
        #run the model
        r2, MSE, pearson = rfr(X = X_train_features, y = y_train_features, test_size=0.2, random_state = seeds[s], iterations=3)
        print('R-squared: '+str(r2)+'\n'+'Mean squared error: '+str(MSE))
    

# feature selection with permutation_importance

read in feature_importances data and test on this random forest method to see if they agree

In [17]:
#read in the feature_importances_ feature selected data
feature_list = []
with open("feat_select_files/phospho/rfr_X_main/rfr_feat_select.txt", "r") as features:
    lines = features.readlines()
    for i in lines:
        i.replace(" ", "")
        feature_list.append(i.split(":")[0])

In [18]:
#create a new X dataframe with the selected features
X_features = X_phos.reindex(feature_list[:1000],axis="columns")

#produce the other required dataframes
hotdrugsDF_feats = one_hot_maker(y_phos)
cl_phos = clMaker(X_phos, y_phos)
x_all_feats, x_drug_feats, y_main_feats = create_all_drugs(x=X_features, xd=hotdrugsDF_feats, y=y_phos, cells=cl_phos)
X_main_feats = X_main_maker(x_all_feats, x_drug_feats, short = False)

In [19]:
X_train_feats, X_test_feats, y_train_feats, y_test_feats = cell_line_split(X_main_feats, y_main_feats, test_size=0.2, random_state = 0 )

In [ ]:
# permutation importance- fit model
classify = RandomForestRegressor(n_jobs=-1, max_depth=300, n_estimators=200)
classify.fit(X_main_feats.values, y_main_feats)

In [ ]:
# permutation importance
result = permutation_importance(classify, X_main_feats.values, y_main_feats, n_repeats=10, random_state=0)

In [ ]:
#this function outputs the top x number of features and their scores for a model

rfr_final_names, rfr_final_scores = rfrFeatures(result.importances_mean, X_main = X_main_feats, topX = 1000, N = 1000)

In [ ]:
#plot the data

plt.rcParams['figure.figsize'] = [20, 20]
plt.plot(rfr_final_names[:50], rfr_final_scores[:50])
rot = plt.xticks(rotation=45)

# MinMax normalised data

In [ ]:
#initialise the normaliser object
MinMax = MinMaxScaler()

In [ ]:
#normalise the data
X_features_MinMax = MinMax.fit_transform(X_features)
X_features_MinMax  = pd.DataFrame(X_features_MinMax, columns=X_features.columns,index=X_features.index)

In [ ]:
#create the relevant dataframes
hotdrugsDF_feats_MinMax = one_hot_maker(y)
x_all_feats_MinMax, x_drug_feats_MinMax, y_main_feats_MinMax = create_all_drugs(x=X_features, xd=hotdrugsDF_feats_MinMax, y=y, cells=cl)
X_main_feats_MinMax = X_main_maker(x_all_feats_MinMax, x_drug_feats_MinMax, short = False)

In [ ]:
#train-test split
X_train_feats_MinMax, X_test_feats_MinMax, y_train_feats_MinMax, y_test_feats_MinMax = cell_line_split(X_main_feats_MinMax, y_main_feats_MinMax, cl, test_size=0.2, random_state = 0)

In [ ]:
# run the random forest model

rfr(X = X_train_feats_MinMax, y = y_train_feats_MinMax, iterations=3)